# Interference in randomized controlled trials

One failure case for most causal inference methods is *interference* between experimental units. The problem arises when the treatment assignment to one individual affects the the outcomes of other individuals. This could be, for example, the consequence of network effects and interaction between individuals. 

* [Incarceration simulator](#Incarceration-simulator)
* [Civil violence simulator](#Civil-violence-simulator)

## Incarceration simulator

To study this problem we will work with a simulator based on the paper:

* *Lum K, Swarup S, Eubank S, Hawdon J. The contagious nature of imprisonment: an agent-based model to explain racial disparities in incarceration rates. J R Soc Interface. 2014;11(98):20140409. [doi:10.1098/rsif.2014.0409](https://dx.doi.org/10.1098%2Frsif.2014.0409)*

The paper proposes an agent-based model that models incarceration as "contagious" in the sense that social ties to incarcerated individuals lead to a higher risk of being imprisoned. The simulation occurs on a fixed set of agents with a fixed set of social ties. What varies is the randomness with which incarceration is passed on and randomness in sentence length. Transition probabilities, and the sentence length distribution are based on real data. The paper shows that higher-on-average sentence lengths for black individuals than for whites lead to a disparity in incarceration rates that resembles the one observed in the United States.

### A sentence length experiment

From a causal inference perspective, a natural question to ask is: 

* What is the effect of harsher sentences on total incarceration times in the population?

To answer this question, imagine we had a randomized trial where individuals are randomly assigned either a sentence from the "harsh" sentencing distribution or a sentence from the more "lenient" sentence distribution. In fact, in reality researchers are often hoping to recover some form of randomization due to the fact that different judges lead to different sentencing distributions, and the assignment to judges is randomized in certain scenarios.

In [1]:
import numpy as np

from whynot.simulators import incarceration

In [2]:
[sentence_length_experiment] = incarceration.get_experiments()

Let's understand what the experiment returns.

**NOTE: The following cell may take a few minutes to execute.** 

In [3]:
random_seed = 0

# The first argument of .run is ignored as the simulator uses a fixed number of samples.
dataset = sentence_length_experiment.run(None, random_seed)

In [4]:
covariates = dataset.covariates

# Fraction of times a harsh sentence was assigned to each individual under random assignment
treatments = dataset.treatments 

# Time spent in prison for each individual
outcomes = dataset.outcomes

# Time spent in prison for each individual, had only harsh sentences been given
outcomes_harsh_only = dataset.true_effects[0]

# Time spent in prsion for each individual, had only lenient sentences been given
outcomes_lenient_only = dataset.true_effects[1]

In [5]:
print('Number of agents: ', len(treatments))

Number of agents:  8856


Here, `outcomes` is the number of months spent in prison for each individual, when prison sentences are assigned randomly to be either drawn from the lenient distribution or drawn from a harsher distribution. 

The harsh distribution has, per default, a mean of 17 months, while the lenient distribution has a mean of 14 months.

Without interference, we could estimate the relative increase in total prison time due to harsh sentence as follows:

In [6]:
prison_time_treatment = np.sum(np.multiply(treatments, outcomes))
prison_time_control = np.sum(np.multiply(1.0 - treatments, outcomes))
relative_increase = prison_time_treatment / prison_time_control
print("%d, %d, %.2f" % (prison_time_treatment, prison_time_control, relative_increase))

10663, 10303, 1.03


But in our setting, we cannot trust this estimate. The fact that one individual receive a long sentence makes it more likely for individuals in his social environment (parents, siblings, children, friends) to also end up incarcerated. Therefore, we contrast this result with simulating prison time had sentences always been drawn from the harsh distribution, and prison time had sentences always been drawn from the lenient distribution. These counterfactual quantities are provided by `dataset.true_effects`.

In [7]:
prison_time_harsh = np.sum(outcomes_harsh_only)
prison_time_lenient = np.sum(outcomes_lenient_only)
relative_increase = prison_time_harsh / prison_time_lenient
print("%d, %d, %.2f" % (prison_time_harsh, prison_time_lenient, relative_increase))

56218, 11429, 4.92


We can see that the relative increase is vastly higher here. This finding corresponds to the insight of the original study that found that relatively small increases in sentencing length can lead to large differences in incarceration rates as a consequence of interference effects.

**NOTE: The following cell can take several minutes to execute.** Decrease `num_trials` for faster execution.

In [8]:
from multiprocessing import Pool

num_processors = 8
num_trials = 16

def counts(random_seed):
    """Run simulation and return prison time counts for treatment, control, and ground truth."""
    dataset = sentence_length_experiment.run(None ,random_seed)
    return (np.sum(np.multiply(dataset.outcomes, dataset.treatments)),
            np.sum(np.multiply(dataset.outcomes, 1.0 - dataset.treatments)),
            np.sum(dataset.true_effects[0]), np.sum(dataset.true_effects[1]))

prison_times = Pool(num_processors).map(counts, range(num_trials))

In [9]:
print("Relative increase in means under randomized assignment: %.2f" %
     (np.mean([x[0] for x in prison_times])/np.mean([x[1] for x in prison_times])))
print("Relative increase in means of ground truth: %.2f" %
     (np.mean([x[2] for x in prison_times])/np.mean([x[3] for x in prison_times])))

Relative increase in means under randomized assignment: 1.07
Relative increase in means of ground truth: 3.65


## Civil violence simulator

We will experiment with a second agent-based model derived from the "civil violence" model introduced in the work:

*  *Modeling civil violence: An agent-based computational approach. Joshua M. Epstein.
Proc. National Academy of Sciences May 2002, 99 (suppl 3) 7243-7250; DOI: [10.1073/pnas.092080199](https://doi.org/10.1073/pnas.092080199)*


### Risk aversion experiment

The treatment in the risk aversion experiment corresponds to increasing the risk aversion of an individual. The larger the risk aversion the less likely the agent is to participate in a riot.

In [10]:
from whynot.simulators import civil_violence

[risk_aversion_experiment] = civil_violence.get_experiments()

**Note: A single run of `.run` runs the simulator `num_samples` times to get ground truth efffects. This makes the call fairly slow. A call with `num_samples=100` might take around one hour.**

In [11]:
dataset = risk_aversion_experiment.run(100, 1)

In the absence of interference between units, we would be able to estimate the average treatment effect as follows.

In [12]:
active_days_treatment = np.mean(np.multiply(dataset.outcomes, dataset.treatments))
active_days_control = np.mean(np.multiply(dataset.outcomes, 1.0 - dataset.treatments))
active_days_joint_treatment_effect = active_days_treatment - active_days_control
print(active_days_joint_treatment_effect)

-1.11


However, this estimate does not correspond to the causal effect of unilaterally increasing an agent's risk aversion in isolation of the other agents. The more agents are risk averse to begin with, the fewer agents become active. The fewer agents are active, the fewer other agents become active via the contagion dynamic. Hence, we'd espect that the isolated treatment effect of risk aversion is greater in the sense that it lowers the number of active days of an agent by a greater amount.

This is what we can get at via the true effects available in the data set.

In [13]:
active_days_isolated_treatment_effect = np.mean(dataset.true_effects)
print(active_days_isolated_treatment_effect)

-184.53
